<a href="https://colab.research.google.com/github/AbhiRathore/llmtasks/blob/main/FlanTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 re

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import json
import nltk
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq,T5ForConditionalGeneration,Seq2SeqTrainingArguments,Seq2SeqTrainer
from transformers import  T5Model,T5TokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)



##  Reload the original google/flan-t5-small model.

In [ ]:
set_seed(27122023)
model_name='google/flan-t5-small'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=original_model)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Train the model for a Q&A task that takes a context as additional input along with the ques'on. You can use SQuAD dataset (h_ps://rajpurkar.github.io/SQuAD-explorer/ ) or the smaller Topioca dataset (h_ps://mcgill-nlp.github.io/topiocqa/) . Choose an appropriate task prefix/trigger word and justify the choice.

In [ ]:
lenpfQ = 128
tokenLen = 64
BATCH_SIZE = 8

There are 2 options to get data




1.   From the SAQuad explorer
2.   From hugging face hub




In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2023-12-27 11:41:34--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2023-12-27 11:41:35 (289 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [ ]:
trainJsonpath = "./train-v2.0.json"


In [ ]:

def parse_squad2_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        squad_data = json.load(file)

    parsed_data = []

    for article in squad_data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                if not qa["is_impossible"]:
                  answers = qa["answers"][0]["text"]

                parsed_data.append({
                    'context': context,
                    'question': question,
                    'answer': answers
                })

    return parsed_data

In [ ]:
parsed_data = parse_squad2_data(trainJsonpath)

# Create a Dataframe
parsed_data_df = pd.DataFrame(parsed_data)

In [ ]:
parsed_data_df.head()

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s


In [ ]:
class Squad_Dataset(Dataset):
    ''' Using Dataset class of Pytorch to load the data'''
    def __init__(self, tokenizer, dataframe, lenpfQ, tokenLen):
        self.tokenizer = tokenizer
        self.lenpfQ = lenpfQ
        self.tokenLen = tokenLen
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.lenpfQ, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.tokenLen, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "labels": labels,
        }

In [ ]:
train_data, valitation_data = train_test_split(parsed_data_df, test_size=0.2, random_state=42)

train_indexer = RandomSampler(train_data.index)
validation_indexer = RandomSampler(valitation_data.index)

qa_dataset = Squad_Dataset(tokenizer, parsed_data_df, lenpfQ, tokenLen)

trainData_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_indexer)
validataData_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=validation_indexer)

In [ ]:
for batch in trainData_loader:
  print(batch.keys())
  break

dict_keys(['input_ids', 'labels'])


Now above data can be used in the model training. Below I am using another method of directly data SQUAD data from Hugging face

In [ ]:
from datasets import load_dataset
dataset_squad_v2 = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset_squad_v2

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
dataset_squad_v2['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [ ]:
def tokenize_function_squad(example):
  """Add prefix to the sentences, add context, tokenize the text, and set the labels"""
  context = example["context"]
  prefix = "Please answer this question: "
  prompt = [prefix + question   for question in example["question"]]
  example['input_ids'] = tokenizer(prompt,context, padding="max_length",max_length=lenpfQ, truncation=True, return_tensors="pt").input_ids
  #print(example['answers'])
  answer = [" ".join(answer['text']) for answer in example["answers"]]
  #print(answer)
  example['labels'] = tokenizer(text_target = answer, max_length=tokenLen,padding="max_length", truncation=True).input_ids

  return example

In [ ]:
tokenized_datasets = dataset_squad_v2.map(tokenize_function_squad, batched=True)




Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
## Rouge metrics for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [ ]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 5

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
   model=original_model,
   args=training_args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.108900,0.372394,0.128592,0.066571,0.128266,0.128534
2,0.103600,0.366059,0.129791,0.067373,0.129509,0.129702
3,0.106300,0.363656,0.130227,0.067764,0.130017,0.130103


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.108900,0.372394,0.128592,0.066571,0.128266,0.128534
2,0.103600,0.366059,0.129791,0.067373,0.129509,0.129702
3,0.106300,0.363656,0.130227,0.067764,0.130017,0.130103
4,0.103200,0.364214,0.131153,0.068596,0.130932,0.131092
5,0.102100,0.364334,0.131000,0.068493,0.130737,0.130877


TrainOutput(global_step=81450, training_loss=0.12271779546860612, metrics={'train_runtime': 11996.0071, 'train_samples_per_second': 54.318, 'train_steps_per_second': 6.79, 'total_flos': 3.028133085315072e+16, 'train_loss': 0.12271779546860612, 'epoch': 5.0})

In [ ]:
!cp -r results/ /content/drive/MyDrive/llmflan_Tasks/advancedTasks

In [ ]:
# !cp -r /content/drive/MyDrive/llmflan_Tasks/advancedTasks/results .

## Model Inference

Load the weights

In [ ]:
set_seed(27122023)
last_checkpoint = "./results/checkpoint-81000"
tuned_model = AutoModelForSeq2SeqLM.from_pretrained(last_checkpoint, torch_dtype=torch.bfloat16)
tokenizer_cp = AutoTokenizer.from_pretrained(last_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer_cp, model=tuned_model)

Sample Question

In [ ]:
question = "What do you think about the benefit of Artificial Intelligence?"
context = "Artificial Intelligence is a field of computer science that deals with machine learning and deep learning"


In [ ]:
prefix = "Please answer this question: "
inputs = prefix + question
inputs = tokenizer_cp(inputs,context, padding="max_length",max_length=lenpfQ, truncation=True, return_tensors="pt")

outputs = tuned_model.generate(**inputs,max_new_tokens=10,temperature = .9,
            do_sample=True,
            top_k=200,
            top_p=0.7)
answer = tokenizer_cp.decode(outputs[0])


In [ ]:
answer

'<pad> a field of computer science that deals with machine'

## Evaluate the quality of the model

In [ ]:
def predScore(context,question, referenceAnswer = None,metrics = 'rouge'):
  prefix = "Please answer this question: "
  inputs = prefix + question
  inputs = tokenizer_cp(inputs,context, padding="max_length",max_length=lenpfQ, truncation=True, return_tensors="pt")

  outputs = tuned_model.generate(**inputs)

  score_r = None
  score_b = None

  predicted_answer = tokenizer_cp.decode(outputs[0])

  if referenceAnswer:
    print(f"calculating {metrics}")
    if metrics == 'rouge':
      rouge_m = evaluate.load("rouge")
      score_r = rouge_m.compute(predictions=[predicted_answer],
                          references=referenceAnswer['text'])

    elif metrics == 'bleu':
      bleu_m = evaluate.load("google_bleu")
      score_b = bleu_m.compute(predictions=[predicted_answer],
                          references=[referenceAnswer['text']])

    elif metrics == 'both':
      rouge_m = evaluate.load("rouge")
      score_r = rouge_m.compute(predictions=[predicted_answer],
                          references=[referenceAnswer['text']])
      bleu_m = evaluate.load("google_bleu")
      score_b = bleu_m.compute(predictions=[predicted_answer],
                          references=[referenceAnswer])
    print("Context: \n", context)
    print("\n")
    print("Question: \n", question)
    return {
        "Reference Answer: ": referenceAnswer,
        "Predicted Answer: ": predicted_answer,
        "BLEU Score: ": score_b,
        "ROUGE Score: ": score_r
    }
  else:
      return predicted_answer


Taking few examples from validation data

In [ ]:
index = 40
question = tokenized_datasets['validation'][index]['question']
context = tokenized_datasets['validation'][index]['context']
referenceAnswer = tokenized_datasets['validation'][index]['answers']


In [ ]:
predScore(context,question, referenceAnswer,metrics = 'bleu')

calculating bleu
Context: 
 Soon after the Normans began to enter Italy, they entered the Byzantine Empire and then Armenia, fighting against the Pechenegs, the Bulgars, and especially the Seljuk Turks. Norman mercenaries were first encouraged to come to the south by the Lombards to act against the Byzantines, but they soon fought in Byzantine service in Sicily. They were prominent alongside Varangian and Lombard contingents in the Sicilian campaign of George Maniaces in 1038–40. There is debate whether the Normans in Greek service actually were from Norman Italy, and it now seems likely only a few came from there. It is also unknown how many of the "Franks", as the Byzantines called them, were Normans and not other Frenchmen.


Question: 
 Who was the Normans' main enemy in Italy, the Byzantine Empire and Armenia?


{'Reference Answer: ': {'text': ['Seljuk Turks',
   'the Pechenegs, the Bulgars, and especially the Seljuk Turks',
   'the Seljuk Turks'],
  'answer_start': [161, 114, 157]},
 'Predicted Answer: ': '<pad> Pechenegs, the Bulgars, and especially the Seljuk Turks',
 'BLEU Score: ': {'google_bleu': 0.7391304347826086},
 'ROUGE Score: ': None}

The results in few cases are not as per the mark and that is mainly due to the less training . In order to improve the results, we can implement a larger model (instead of t5 small) and also use peft based approaches like LORA. where a separate weight matric (lower order) can be tuned to perform a specific task. Currenty i am running only 5 spochs, due to hardware limitation

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/FlanTuning.ipynb' '/content/drive/MyDrive/llmflan_Tasks/advancedTasks/'